## OpenAI 외의 도구 호출 Agent 만들기(Tool Calling Agent with others)

OpenAI 외에도 Langchain은 `Anthropic`, `Google Gemini`, `Mistral`, `Together.ai`, `Ollama`과 같은 여러 공급자 구현을 지원한다. 

여기서는 다양한 LLM으로 도구 호출 에이전트를 생성하고 실행하는 방법을 살펴보자.

Source : https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/

In [24]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging
logging.langsmith("agent-other-llm")

LangSmith 추적을 시작합니다.
[프로젝트명]
agent-other-llm


In [25]:
from langchain.tools import tool 
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews

# Define tool
@tool
def search_google_news(query: str)-> List[Dict[str, str]] :
    """Search google news by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


print(f"도구 이름 : {search_google_news.name}")
print(f"도구 설명 : {search_google_news.description}")


도구 이름 : search_google_news
도구 설명 : Search google news by input keyword


In [26]:
# Define tool
tools = [search_google_news]

### Agent 용 프롬프트 생성

- `chat_history` : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- `agent_scratchpad` : 에이전트가 임시로 저장하는 변수
- `input` : 사용자의 입력

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

# create prompt
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공한다 (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant that can search google news by input keyword"
     ),
     ("placeholder", "{chat_history}"),
     ("human", "{input}"),
     ("placeholder", "{agent_scratchpad}"),
])

### Tool Calling 을 지원하는 LLM List

실습 진행을 위해 아래 내용을 설정한다.
- Anthropic
    - [Anthropic API 키 발급 관련](https://console.anthropic.com/settings/keys)
    - .env 파일 내 ANTHROPIC_API_KEY 에 발급받은 키 설정
- Gemini
    - [Gemini API 키 발급 관련](https://aistudio.google.com/app/apikey?hl=ko)
    - .env 파일 내 GOOGLE_API_KEY 에 발급받은 키 설정
- Together AI
    - [Together AI API 키 발급 관련](https://api.together.ai/signin)
    - .env 파일 내 TOGETHER_API_KEY 에 발급받은 키 설정
- Ollama
    - [Ollama Tool Calling 지원 모델 리스트](https://ollama.com/search?c=tools)
    - [이번 실습에 사용할 llama3.1 모델](https://ollama.com/library/llama3.1)
    - 터미널 창에 ollama pull llama3.1 명령어를 입력하여 모델 다운로드
    - 이전에 Ollama 를 사용하지 않았다면, [Ollama](https://wikidocs.net/233805) 를 참고한다.

langchain-ollama 설치를 한 뒤 진행한다.


In [28]:

# ! pip install -qU langchain-ollama==0.1.3
# ! pip install -U notebook

In [29]:
# ! pip install -q langchain_anthropic langchain_google_vertexai langchain_ollama

In [30]:
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# Claude 3.5 Sonnet
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# Gemini 1.5 Pro
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
# gemini = ChatVertexAI(model="gemini-1.5-pro", temperature=0)

# TogetherAI : Llama-3.1-70B-Instruct-turbo
llama = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
)

# # Llama 3.1
ollama = ChatOllama(model="llama3.1", temperature=0)

# # Qwen2.5 7B (한글성능 나쁘지 않음.)
qwen = ChatOllama(model="qwen2.5:latest")

각 LLM 기반으로 Agent를 생성한다.

In [31]:
from langchain.agents import create_tool_calling_agent

gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
claude_agent = create_tool_calling_agent(claude, tools, prompt)
gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
llama_agent = create_tool_calling_agent(llama, tools, prompt)
ollama_agent = create_tool_calling_agent(ollama, tools, prompt)
qwen_agent = create_tool_calling_agent(qwen, tools, prompt)


gemini는 SDK를 설치해야한다고 하니 그냥 넘어가자.

### AgentExecutor 생성 후ㅜ 실행 및 결과 확인

In [32]:
from langchain.agents import AgentExecutor

# gpt_agent 실행
agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

result = agent_executor.invoke ({
    "input": "미국 AI 투자와 관련된 뉴스를 검색해주세요."
})

print("Agent 실행 결과 : \n", result["output"])





> Entering new AgentExecutor chain...

Invoking: `search_google_news` with `{'query': '미국 AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiXkFVX3lxTFA3WU9zaU16TlNPWEVVMU1DaHVGQVhTeXhUNzZjSG5RVnRNbEo0Q2duV2JZdlhLOUdCUi1FVEMxVzZRSHVsX1ZlSTdMWUs2dmtORmFrMnZoYzYtVHBzeFHSAXZBVV95cUxPY24wZ05Td0JqSmRsSUZUOG1NZUtmTEdxTTI2aTV3d1RfbHdoYkFDQ1N2MnUtWWJjZmduTUI5RHBGRUs3NFJfUzd0TEgwSHlLSThneWdqSHNNV0tzcHZaazlFUTB3Rm12SG9UVGNaUGlqRm4zVHZR?oc=5', 'content': "트럼프, 'AI 패권' 야심 드러내...중국 노골적 견제 - YTN"}, {'url': 'https://news.google.com/rss/articles/CBMiUEFVX3lxTE05TlIzaS0wSlN3V3NFZ0xtTnh1aVJYOWdSSVowQlE2YkZaaklTQk5IOF9FTE0xSDZYV25RYkV4dFl5RWVYQmFjbHJKMnV6RVQt0gFTQVVfeXFMTnFTNks0eXlSX0NESm1JX1hoOG1MbnlRcnM0ZjEtTDNOeTdxR0V6RVgxdmZEWXpOWGRNMzhSS1ZzblBzcEYwQl8yYlhQSjJ6bW1NMmc?oc=5', 'content': '트럼프, AI 분야 ‘5천억 달러’ 투자 계획 발표 - VOA Korea'}, {'url': 'https://news.google.com/rss/articles/CBMihAFBVV95cUxQN3FmTE9KZ2llMzVFa0dSQUdySlUyLU9DNTliaUgtN2hNSEd2SkRqYVJkUUoxOVNRbThOZ0dOY2FSZWpaTVZTaE9wTjN0UXZMLS

이제 다양한 llm을 사용하여 agent를 실행해보자.

다음은 입력받은 llm을 사용하여 Agent를 생성하고 실행하여 결과를 출력하는 함수이다.

In [33]:
def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[ '{label}' 실행 결과 ]")

    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result ["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])


각 LLM 별로 에이전트를 생성, 실행하여 결과를 출력.

In [34]:
query = (
    "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요."
)

In [35]:

# gpt
execute_agent(gpt, tools, query, "gpt")

[ 'gpt' 실행 결과 ]
🌟 **AI 투자 동향** 🌟

1️⃣ **MS와 메타**: 메타가 AI에 대한 지속적인 투자를 통해 주가 상승세를 이어가고 있다는 소식입니다! 🤑 [자세히 보기](https://news.google.com/rss/articles/CBMiWkFVX3lxTE8yeEdfOWxJSmtHSUJYS0ZXVDJaU1RWUTNzU01EVkJQaFpOREZNRE95REpjWU9YR3ZJTlAyODJVdzU2djNZSW5kUjRpVkdid3lfYU9USlNlLTRuZ9IBVEFVX3lxTE1xVzJZZ1lRQ3R6UnlwYkkwS1dsSVBWV2o5TmZoTjBDMGtRanZzMENMMlE2bE9PbHpGMjVsQ0ZOYlk1Wk96bWJGMzlkQ3pqVVZvTndmcw?oc=5)

2️⃣ **트럼프 발 AI 투자**: AI 투자가 한국 기업에 새로운 기회를 제공할 수 있다는 전망이 나오고 있습니다. 전력 솔루션 시장이 주목받고 있네요! 🔍 [자세히 보기](https://news.google.com/rss/articles/CBMiW0FVX3lxTE4zbUlUMW5PZ2xBWVBXQjRrdWM1ZlRlN3BoWE1VRkd3cHRlcTVKSEEtLUZrMWlKc2NkRFY5dmJ3YmlmUTctd2Q4SzNxQnJBZnJMYzJxOVo1WjB1em_SAV5BVV95cUxQWjVzN2t0ejY2akhjZGVGbnpRRWw3ZWUwckFLc3U3NXkwaFcxaTVCRWJHY0M0STNhMU1heURwWmlXa285YnJqa2h5NFdkamVEejhwQnpVNE9uVS1aWk9B?oc=5)

3️⃣ **ETF와 AI 투자**: 금리 하락기에는 ETF와 AI 투자의 조합으로 전략적 자산 배분 가이드가 강조되고 있습니다! 📊 [자세히 보기](https://news.google.com/rss/articles/CBMiVkFVX3lxTFA4WnRkSE1kZnUteWhMdmJWSWFEWUs3Y0tTTVFjc0M2YmNNOEM5b0RHa3pDN0dneXp6ZG9tc

In [36]:
# claude
# execute_agent(claude, tools, query, "claude")

claude는 credit balance 가 부족하여 실행이 안되니 그냥 넘어가자.

In [37]:
# google gemini
execute_agent(gemini, tools, query, "gemini")

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents[2].parts[0].function_response.name: Name cannot be empty.


In [38]:

# llama3.1 70B (Together.ai)
execute_agent(
    llama,
    tools,
    "Search AI related news and write it in Instagram post format",
    "llama3.1 70B",
)

[ 'llama3.1 70B' 실행 결과 ]
"Hey everyone! 

Want to stay up-to-date on the latest AI news? 

Here are some recent articles from Google News:

1. "딥시크가 '챗GPT' 베낀 증거 있어"..."오픈AI, 합성데이터 활용 방지 예고" - AI타임스
2. "오픈AI 등 ″딥시크 조사…데이터 무단 수집 여부″" - 연합뉴스TV
3. "AI로 제조업 안전사고 예방한다…산업부, R&D 사업 공모" - 연합뉴스
4. "산업부, 제조안전고도화기술개발사업 공고...AI로 제조업 안전사고 예방한다" - AI타임스
5. "AI로 제조업 산재 위험 예방…산업부, 배터리·석유화학 분야 과제 공모" - 뉴시스

Check out these articles to learn more about the latest developments in AI!

#AI #GoogleNews #LatestNews"


In [39]:
# llama3.1 8B (ollama)
execute_agent(ollama, tools, query, "llama3.1(Ollama)")


[ 'llama3.1(Ollama)' 실행 결과 ]
**AI 투자 뉴스**

*   **태국 BOI, 5억 달러 규모의 투자 승인** 태국 BOI는 TikTok의 데이터 호스팅 프로젝트를 포함한 총 5억 달러 규모의 투자를 승인했다. 또한 바이오 경제에 대한 혁신적인 인센티브를 발표했다.
*   **42조 원 규모의 탄약, 'AI' 전부** IT 조선은 42조 원 규모의 탄약이 모두 AI 관련 사업이라고 보도했다.
*   **트럼프 대통령, 미국 내에서의 AI 관련 사업에 77조 엔 설비 투자 발표** nhk.or.jp는 트럼프 대통령이 미국 내에서의 AI 관련 사업에 77조 엔 규모의 설비 투자를 발표했다고 보도했다.
*   **투스 시큐리티, 2025년 미국 투자 전망 발표** KIPOST는 투스 시큐리티가 2025년 미국 투자 전망을 발표했다고 보도했다.
*   **외식업 댓글 관리 AI' 르몽, 블루포인트에서 시드투자 유치** 스타트업엔(StartupN)은 외식업 댓글 관리 AI인 르몽이 블루포인트에서 시드 투자를 유치했다고 보도했다.


In [40]:

# qwen2.5 7B (ollama)
query = "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요. 한글로 답변하세요!"

execute_agent(qwen, tools, query, "qwen2.5(Ollama)")

[ 'qwen2.5(Ollama)' 실행 결과 ]
Instagram 게시글:

🚀 AI 투자 뉴스 🚀

1. MS, 메타 "AI 투자 지속"에 추가 상승 달렸다 [김현석의 월스트리트나우] - 한국경제

2. 트럼프발 AI 투자, 한국기업에 기회?… "전력 설루션 시장 주목" - 연합뉴스

3. ETF·AI 투자의 새 물결… 금리 하락기의 전략적 자산 배분 가이드 [AI PRISM*주간 금융상품 투자자 뉴스] - 서울경제신문

4. AI 투자 적게 했다는 이유로 애플은 3% 급등, 시총 1위 탈환 - 뉴스1

5. 어닝서프라이즈 메타 4% 급등.. “AI인프라 투자 계속” - 매일경제

# 참고 링크: 
1. [MS, 메타 "AI 투자 지속"에 추가 상승 달렸다](https://news.google.com/rss/articles/CBMiWkFVX3lxTE8yeEdfOWxJSmtHSUJYS0ZXVDJaU1RWUTNzU01EVkJQaFpOREZNRE95REpjWU9YR3ZJTlAyODJVdzU2djNZSW5kUjRpVkdid3lfYU9USlNlLTRuZ9IBVEFVX3lxTE1xVzJZZ1lRQ3R6UnlwYkkwS1dsSVBWV2o5TmZoTjBDMGtRanZzMENMMlE2bE9PbHpGMjVsQ0ZOYlk1Wk96bWJGMzlkQ3pqVVZvTndmcw?oc=5)
2. [트럼프발 AI 투자, 한국기업에 기회?… "전력 설루션 시장 주목"](https://news.google.com/rss/articles/CBMiW0FVX3lxTE4zbUlUMW5PZ2xBWVBXQjRrdWM1ZlRlN3BoWE1VRkd3cHRlcTVKSEEtLUZrMWlKc2NkRFY5dmJ3YmlmUTctd2Q4SzNxQnJBZnJMYzJxOVo1WjB1em_SAV5BVV95cUxQWjVzN2t0ejY2akhjZGVGbnpRRWw3ZWUwckFLc3U3NXkwaFcxaTVCRWJHY0M0STNhMU1heURwWmlXa285YnJqa2h5NFdkamVEejhwQnpVNE9uVS1aWk9B?oc